# Assignment week 7

## Notebook made by   (If not filled in correctly: 0 pts for assignment)

__Name(s)__: Micha de Groot & Kaj Meijer

__Student id(s)__ : 10434410 & 10509534

### Pledge (taken from [Coursera's Honor Code](https://www.coursera.org/about/terms/honorcode) )



Put here a selfie with your photo where you hold a signed paper with the following text: (if this is team work, put two selfies here). The link must be to some place on the web, not to a local file. **Assignments without the selfies will not be graded and receive 0 points.**

> My answers to homework, quizzes and exams will be my own work (except for assignments that explicitly permit collaboration).

>I will not make solutions to homework, quizzes or exams available to anyone else. This includes both solutions written by me, as well as any official solutions provided by the course staff.

>I will not engage in any other activities that will dishonestly improve my results or dishonestly improve/hurt the results of others.

[Selfie](http://imgur.com/RiMa4YY)

 
In this task we will try to expand the information that we find in the infoboxes using WordNet.

We follow the idea described in the YAGO paper: a string like `a beautiful actress from Denmark` is chopped up in a pre-modifier, a head and a post modifier. Here the head obviously is `actress`. 

## The tasks

 We perfom the task on a spreadsheet containing information about almost 1M persons found in the english Wikipedia.
In particular we are interested in the  `description` column.

1. (20%) For each string in the `description` column do the following:
    1. Determine pre-modifier,  head and  post modifier: store these in new columns
2.  (20%) For each head, determine the list of all ancestors of that head in WordNet up to "person": store this list in a new column.
3. (20%) Use Hearst patterns to extract something like the `description` from the `rdf-schema#comment`
    * Add your version as a new column
    * Design a comparison of your work with the description column which outputs something like accuracy/precision/recall. You are free, as long as you do it well.
4. (20%) Use Hearst patterns plus whatever else you like to extract the nationatility of the person described.
    * Include a brief evaluation.
    
2(actually 5). (20%) We are interested in gender aspects. Can you answer these questions:
    1. Can we determine the gender from the person from the description and its expansion? 
        * E.g., we could say it is a female if it was actress, but probably not say it is a male if it was actor
    2. There are two columns indicating the gender of the persons. One is made based on the first name, the other on the 3rd person pronouns in the longer description. Both are fairly accurate (more than .9 precision). Can you find a couple of remarkable differences between males and females based on your extracted wordnet concepts?

## Data
Available at <http://maartenmarx.nl/teaching/KennisSystemen/Data/PersonDF_naam_sex_description_snippet.csv.gz>
    
## Hints
* Do this in a python notebook, using pandas, and possibly the pattern module for parsing the description.
* Make a 10K version of the file and experiment with that first.

In [46]:
# Opgave 1
from pattern.en import pprint, parse
import csv
import re 


with open('new2.csv', 'rb') as csvfile:
    csvReader = csv.reader(csvfile, delimiter='\t')
    for row in csvReader:
        splitRow = row[0].split(',')[3]
        splitRow = parse(splitRow).split(' ')
        print splitRow
        pre = []
        head = []
        post = []
        for word in splitRow:
            if re.match('.*/NN(P)?/.*', word):
                #print word, "NN"
                head.append(word)
            if re.match('.*/JJ/.*', word):
                #print word, "JJ"
                pre.append(word)
            if not re.match('.*/O', word):
                #print word, "Not O"
                post.append(word)
        print "Pre: ", pre
        print "Head: ", head
        print "post: ", post

[u'Geographer/NN/B-NP/O', u'Photographer/NNP/I-NP/O']
Pre:  []
Head:  [u'Geographer/NN/B-NP/O', u'Photographer/NNP/I-NP/O']
post:  []
[u'']
Pre:  []
Head:  []
post:  [u'']
[u'Potential/JJ/B-NP/O', u'heiress/NN/I-NP/O', u'to/TO/O/O', u'the/DT/B-NP/O', u'Danish/JJ/I-NP/O', u'throne/NN/I-NP/O']
Pre:  [u'Potential/JJ/B-NP/O', u'Danish/JJ/I-NP/O']
Head:  [u'heiress/NN/I-NP/O', u'throne/NN/I-NP/O']
post:  []
[u'Queen/NNP/B-NP/O', u'consort/NN/I-NP/O', u'of/IN/B-PP/B-PNP', u'France/NNP-LOC/B-NP/I-PNP']
Pre:  []
Head:  [u'Queen/NNP/B-NP/O', u'consort/NN/I-NP/O']
post:  [u'of/IN/B-PP/B-PNP', u'France/NNP-LOC/B-NP/I-PNP']
[u'Queen/NNP/B-NP/O', u'consort/NN/I-NP/O', u'of/IN/B-PP/B-PNP', u'Norway/NNP-LOC/B-NP/I-PNP']
Pre:  []
Head:  [u'Queen/NNP/B-NP/O', u'consort/NN/I-NP/O']
post:  [u'of/IN/B-PP/B-PNP', u'Norway/NNP-LOC/B-NP/I-PNP']
[u'Abbess/NNP/B-NP/O', u'of/IN/B-PP/B-PNP', u'Vadstena/NNP/B-NP/I-PNP', u'Abbey/NNP/I-NP/I-PNP']
Pre:  []
Head:  [u'Abbess/NNP/B-NP/O', u'Vadstena/NNP/B-NP/I-PNP', u'

In [1]:
# Opgave 2
from nltk.corpus import wordnet as wn

head = wn.synset('lawyer.n.01')
hypernym = head.hypernyms()
print hypernym
while(str(hypernym) != "[Synset('person.n.01')]"):
    hypernym = hypernym[0].hypernyms()
    print hypernym

[Synset('professional.n.01')]
[Synset('adult.n.01')]
[Synset('person.n.01')]


In [105]:
# Opgave 3
from pattern.en import pprint, parse
import csv
import re 

'''
This code analyses the comment column of the csv file and 
looks for a Hears pattern of the form X is Y/X was Y.
After that it compares the found Y to the original description of the csv file.
For the comparison the F1 score is used. It is observed that 
the newley generated descriptions is almost always longer than
the original.
This results in a high recall and low precision on average.
Smoothing with the F1 score tis therefore always necessary.
'''

def hearstPattern(text):
    #text = text.split('.')[0]
    if re.match('.* was .*', text):
        text = re.split(" was ", text)
        if re.match('.* is .*', text[0]):
            text = re.split(" is ", text[0])
            text = text[1].split('.')[0]
            # print "Desc: ", text, "\n"
        else:
            text = text[1].split('.')[0]
            # print "Desc: ", text, "\n"
    elif re.match('.* is .*', text):
        text = re.split(" is ", text)
        text = text[1].split('.')[0]
        # print "Desc: ", text, "\n"
    return text

# For the comparison we use the F1 score. 
def compare(description1, description2):
    description1 = description1.lower().split()
    description2 = description2.lower().split()
    if( len(description1) == 0 or len(description2) == 0):
        return 0
    truepositives = 0.0
    for word in description1:
        if word in description2:
            truepositives +=1
    recall = truepositives / len(description1)
    precision = truepositives / len(description2)
    #print "Recall: ", recall
    #print "Precision: ", precision
    if precision == 0 and recall == 0:
        return 0
    
    F1score = 2*(precision*recall)/(precision+recall)
    return F1score

with open('new3.csv', 'rb') as csvfile:
    csvReader = csv.reader(csvfile, delimiter='\t')
    for row in csvReader:
        comment = row[0].split(',')[5]
        originalDescription = row[0].split(',')[3]
        print originalDescription
        pattern = hearstPattern(comment)
        print pattern
        similarity = compare(originalDescription, pattern)
        print similarity, "\n"

abbess of the Bridgettine Abbey of Vadstena
a Swedish nun and noble the first official abbess of the Bridgettine Abbey of Vadstena in 1385/88–1403
0.583333333333 

Swedish princess and a Grand Princess of Kiev
a Swedish princess and a Grand Princess of Kiev
0.941176470588 

swedish politician
a Swedish Green Party politician
0.571428571429 

Swedish politician
a Swedish Moderate Party politician
0.571428571429 


a Swedish-Norwegian royal servant
0 

Canoe racer
a Swedish sprint canoer who competed from the mid-1990s to the early 2000s (decade)
0 

Swedish songwriter
a Swedish lyricist in popular music
0.25 

Swedish privateer
a Swedish privateer in service of King Charles XII of Sweden during the Great Northern War
0.222222222222 

Actress
a Swedish actress
0.5 

Norwegian politician
a Norwegian civil servant and politician
0.5 

Swedish author(children books)
a Swedish author
0.333333333333 

Norwegian police chief
a Norwegian jurist and the current Police Commissioner in the Nationa

In [123]:
# Opgave 4

# Opgave 3
from pattern.en import pprint, parse
import csv
import re 

'''
Looks in the comments for the first descriptive sentence. 
The observed pattern is that each sentence that contains
'was a(n)' or 'is a(n)' usually has the nationality in that 
sentence. It is assumed that the first capitalized
word is the nationality.
This works for almost all cases that actually contain a
nationality in the comment
'''

def hearstPattern(text):
    #text = text.split('.')[0]
    if re.match('.* was .*', text):
        text = re.split(" was ", text)
        if re.match('.* is .*', text[0]):
            text = re.split(" is ", text[0])
            text = text[1].split('.')[0]
            # print "Desc: ", text, "\n"
        else:
            text = text[1].split('.')[0]
            # print "Desc: ", text, "\n"
    elif re.match('.* is .*', text):
        text = re.split(" is ", text)
        text = text[1].split('.')[0]
        # print "Desc: ", text, "\n"
    return text

def findNationality(text):
    text = text.split()
    if text[0] == 'a' or text[0] == 'an':
        for i in range(len(text)):
            if text[i][0].isupper():
                return text[i]
    return None

with open('new2.csv', 'rb') as csvfile:
    csvReader = csv.reader(csvfile, delimiter='\t')
    for row in csvReader:
        comment = row[0].split(',')[5]
        originalDescription = row[0].split(',')[3]
        print originalDescription
        pattern = hearstPattern(comment)
        print "C: ", comment
        print "P: ", pattern
        nationality = findNationality(pattern)
        print "N: ", nationality, "\n"


Geographer Photographer
C:  Ingeborg de Beausacq (January 25 1910 – July 12 2003) was an American photographer and explorer of German origin
P:  an American photographer and explorer of German origin
N:  American 


C:  Ingeborg Danielsdotter i Mjärhult (1665 – 23 July 1749) was a Swedish natural healer medicine woman herbalist natural philosopher soothsayer and spiritual visionary. She was one of the most famous of the many so-called Cunning folk of her time in her country's history.
P:  a Swedish natural healer medicine woman herbalist natural philosopher soothsayer and spiritual visionary
N:  Swedish 

Potential heiress to the Danish throne
C:  Ingeborg of Denmark (Danish: Ingeborg Valdemarsdatter) (1 April 1347–16 June 1370) was the eldest daughter of Valdemar IV of Denmark and his wife Helvig of Schleswig. By marriage she was Duchess of Mecklenburg-Schwerin although she died before her husband's succeed her father-in-law.
P:  the eldest daughter of Valdemar IV of Denmark and his w

Opgave 5.1

It is possible to detemine the gender of the person from the description if it contains gender specific terms. Most of the gender specific terms are female terms like actress, where actor could be male of female. 

Opgave 5.2

!!! We need data for this. !!!

In [8]:
# http://www.clips.ua.ac.be/pages/pattern-en#parser
>>> from pattern.en import parse
>>> print parse('a beautiful actress from Denmark walks')

a/DT/B-NP/O beautiful/JJ/I-NP/O actress/NN/I-NP/O from/IN/B-PP/B-PNP Denmark/NNP-LOC/B-NP/I-PNP walks/VBD/B-VP/O


In [9]:
from pattern.en import pprint, parse
pprint(parse('a beautiful actress from Denmark walks', lemmata=True))

          WORD   TAG       CHUNK   ROLE   ID     PNP    LEMMA       
                                                                    
             a   DT        NP      -      -      -      a           
     beautiful   JJ        NP ^    -      -      -      beautiful   
       actress   NN        NP ^    -      -      -      actress     
          from   IN        PP      -      -      PNP    from        
       Denmark   NNP-LOC   NP      -      -      PNP    denmark     
         walks   VBD       VP      -      -      -      walk        


In [6]:
pprint(parse('Ingeborg Moen Borgerud  is a Norwegian lawyer businessperson and former politician for the Labour Party.'))

Pretty printing has been turned OFF


In [8]:
# read 

import pandas as pd
Pdf = pd.read_csv('http://maartenmarx.nl/teaching/KennisSystemen/Data/PersonDF_naam_sex_description_snippet.csv.gz',
                 compression='gzip')
Pdf = Pdf[Pdf.columns[2:]]
print Pdf.shape

Pdf.head()

(969996, 7)


,rdf-schema#label,description,gender,rdf-schema#comment,gender_from_pronouns,comment_length,word_comment_length
0,Ingeborg Moen Borgerud,Norwegian businesswoman and politician,U,Ingeborg Moen Borgerud (born 23 September 1949...,NaN,128,19
1,Ingeborg Nilsson,Figure skater,U,Ingeborg Nilsson (1924 – 1995) was a Norwegian...,female,247,41
2,Ingeborg Norell,Finnish heroine / life saver,U,Ingeborg Norell (born 1727) was the first Finn...,female,242,41
3,Ingeborg Pehrson,Danish actor,U,Ingeborg Pehrson (16 December 1886 - 11 April ...,NaN,88,16
4,Ingeborg Pfüller,Argentine shot putter and discus thrower,U,Ingeborg Ella Pfüller (born January 1 1932) wa...,female,301,52


In [78]:
# add some columns to Pdf 

# in this simple example we add the number of tokens in the description as a new column

def telwoordjes(string):
    return len(string.split())

Pdf['Description_length']= Pdf.description.astype(str).apply(telwoordjes)

Pdf.Description_length.head()

NameError: name 'Pdf' is not defined

In [14]:
# save, gzip, and load again

%time Pdf.to_pickle('PersonDF_naam_sex_description_snippetEXTRA.pkl')

%ls -lh 'PersonDF_naam_sex_description_snippetEXTRA.pkl'
%time !gzip PersonDF_naam_sex_description_snippetEXTRA.pkl
%ls -lh PersonDF_naam_sex_description_snippetEXTRA.pkl.gz
%time !gunzip PersonDF_naam_sex_description_snippetEXTRA.pkl.gz
# read 

%time Pdf = pd.read_pickle('PersonDF_naam_sex_description_snippetEXTRA.pkl')

!rm PersonDF_naam_sex_description_snippetEXTRA.pkl

CPU times: user 3.96 s, sys: 736 ms, total: 4.7 s
Wall time: 4.72 s
-rw-r--r--@ 1 admin  staff   351M Jul 28 16:24 PersonDF_naam_sex_description_snippetEXTRA.pkl
CPU times: user 91.9 ms, sys: 26.2 ms, total: 118 ms
Wall time: 22.8 s
-rw-r--r--@ 1 admin  staff   124M Jul 28 16:24 PersonDF_naam_sex_description_snippetEXTRA.pkl.gz
CPU times: user 8.9 ms, sys: 14.2 ms, total: 23.1 ms
Wall time: 1.95 s
CPU times: user 3.25 s, sys: 574 ms, total: 3.82 s
Wall time: 3.85 s


In [11]:
# inspect the full descriptions

for x in Pdf['rdf-schema#comment'][:10]:
    print x

Ingeborg Moen Borgerud (born 23 September 1949) is a Norwegian lawyer businessperson and former politician for the Labour Party.
Ingeborg Nilsson (1924 – 1995) was a Norwegian figure skater. She competed at the 1952 Winter Olympics in Oslo.She was Norwegian individual champion in 1953 and 1954 and in pairs in 1956 1957 1958 and 1959 together with partner Reidar Børjeson.
Ingeborg Norell (born 1727) was the first Finnish female to have received an official decoration and to have been the subject of a commemorative plaque. She was commemorated in recognition for her act which saved many lives in Tenala in 1780.
Ingeborg Pehrson (16 December 1886 - 11 April 1950) was a Danish stage and film actress.
Ingeborg Ella Pfüller (born January 1 1932) was a female track and field athlete from Argentina competing in the discus throw. She represented her native country at the 1952 Summer Olympics and won the gold medal in the women's discus throw event at the 1955 Pan American Games in Mexico City M